My working version of time2vec

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from time2vec import Time2Vec

torch.manual_seed(1)

In [2]:
x = list(range(1, 7000))
y = list()

for item in x:
    if item % 7 == 0:
        y.append(1)
    else:
        y.append(0)

df = pd.DataFrame(list(zip(x, y)), columns=["x", "y"])
df.shape

(6999, 2)

In [3]:
df.head(10)

,x,y
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,1
7,8,0
8,9,0
9,10,0


In [4]:
df.tail(10)

,x,y
6989,6990,0
6990,6991,0
6991,6992,0
6992,6993,1
6993,6994,0
6994,6995,0
6995,6996,0
6996,6997,0
6997,6998,0
6998,6999,0


In [5]:
class Time2Vec(nn.Module):
    def __init__(self,in_features, out_features, f=torch.sin, arg=None):
        super(Time2Vec, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.f = f
        self.arg = arg
        
    def forward(self, tau):
        if self.arg:
            v1 = self.f(torch.matmul(tau, self.w) + self.b, self.arg)
        else:
            v1 = self.f(torch.matmul(tau, self.w) + self.b)
        v2 = torch.matmul(tau, self.w0) + self.b0
        return torch.cat([v1, v2], 1)

In [6]:
t2v = Time2Vec(1, 64)

In [7]:
class Model(nn.Module):
    def __init__(self,activation, hidden_dim):
        super(Model, self).__init__()
        if activation == "sin":
            self.l1 = Time2Vec(1, hidden_dim, f=torch.sin,arg=None)
        elif activation == "cos":
            self.l1 = Time2Vec(1, hidden_dim, f=torch.cos,arg=None)
        self.fc1 = nn.Linear(hidden_dim, 2)
    
    def forward(self, x):
        x = self.l1(x)
        x = self.fc1(x)
        return x

In [8]:
class ExampleDataset(Dataset):
    def __init__(self):
        super(ExampleDataset, self).__init__()
        self.x = df["x"].values
        self.y = df["y"].values

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return np.array(self.x[idx]), self.y[idx]


dataset = ExampleDataset()

In [9]:
model = Model(activation="sin", hidden_dim=64)
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
num_epochs = 100
dataloader = DataLoader(dataset, batch_size=2048, shuffle=False)

In [10]:
for ep in range(num_epochs):
    for x, y in dataloader:
        optimizer.zero_grad()
        y_pred = model(x.unsqueeze(1).float())
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
    print("epoch: {}, loss:{}".format(ep, loss.item()))

epoch: 0, loss:13.099471092224121
epoch: 1, loss:10.10128402709961
epoch: 2, loss:7.135891914367676
epoch: 3, loss:4.217010498046875
epoch: 4, loss:1.3543992042541504
epoch: 5, loss:2.147102117538452
epoch: 6, loss:0.9765524864196777
epoch: 7, loss:1.4988539218902588
epoch: 8, loss:1.421852469444275
epoch: 9, loss:0.8869450688362122
epoch: 10, loss:0.4270593225955963
epoch: 11, loss:0.5099457502365112
epoch: 12, loss:0.8238770961761475
epoch: 13, loss:0.8916766047477722
epoch: 14, loss:0.5454902052879333
epoch: 15, loss:0.4697878956794739
epoch: 16, loss:0.4851878881454468
epoch: 17, loss:0.5600579977035522
epoch: 18, loss:0.42498961091041565
epoch: 19, loss:0.41304075717926025
epoch: 20, loss:0.47801411151885986
epoch: 21, loss:0.49191614985466003
epoch: 22, loss:0.4344368875026703
epoch: 23, loss:0.41382336616516113
epoch: 24, loss:0.46551626920700073
epoch: 25, loss:0.44085243344306946
epoch: 26, loss:0.41832396388053894
epoch: 27, loss:0.4436555504798889
epoch: 28, loss:0.441447049